<a href="https://colab.research.google.com/github/RahulJuluru2/Unit1Assignments/blob/main/U1_MH2_AptitudeClassification_V4_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Problem Statement

The problem is to identify the subcategory and classify the question based on the group it belongs to.



## Learning Objectives

At the end of the experiment, you will be able to understand:

*   Beautiful Soup
*   Use NLTK package
*   Text Representation
*   Classification

## Dataset
Being able to classify the questions will be difficult in natural language processing. The dataset is taken from the TalentSprint aptitude questions which contains more than 20K questions.

## Description
This dataset has the following columns:
1. **Category:** Gives the high-level categorization of the question
2. **Sub-Category:** Determines the type of questions
3. **Article:** Gives the article name of the question
4. **Questions:** Questions are listed
5. **Answers:** Contains answers


### Grading = 20 Marks

In [1]:
#@title Download the datasets
from IPython import get_ipython

ipython = get_ipython()

def setup(): 
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Aptitude_Classification_data.csv")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Mentors_Test_Data.csv")
    from IPython.display import HTML, display
    print("Setup completed successfully")
    return

setup()

Setup completed successfully


In [2]:
# Import Python Libraries
from bs4 import BeautifulSoup
import nltk
import re
import string
import warnings
import numpy as np
import pandas as pd
from collections import Counter
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

##   **Stage 1**:  Dataset Preparation

### 1 Mark -> Load the data set and prepare the data based on group allocation. 

Each group should consider their respective sub-categories as mentioned below:

> Team A = Groups 1, 4, 7, 10;   &nbsp; &nbsp; Sub-Category = Misspell words, Algebra, Percentages, Mathematical operations, Probability

> Team B = Groups 2, 5, 8, 11; &nbsp; &nbsp;   Sub-Category = Finding Errors, Ratio and Proportion, Logarithms, Time and Distance, Simple and Compound Interest

> Team C = Groups 3, 6, 9, 12;  &nbsp; &nbsp;  Sub-Category =  Synonyms and Antonyms, Time and Work, Permutations and Combinations, LCM and HCF, Profit and Loss


**Hint:** &nbsp; To access Sub-Categories from given Data, refer [link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)

In [3]:
# Read the excel file and read data
aptitude_questions_bank_df = pd.read_csv('Aptitude_Classification_data.csv')
with pd.option_context("display.max_columns",None):
  display(aptitude_questions_bank_df.head(2))

,Category,Sub-Category,Article,Questions,Answers
0,Verbal,Misspell words,chapter 1,Which of the following is correct?\n\n\n\n\n,2
1,Quantitative,Time and Distance,Time and Distance - Model 05,Rohan leaves point A and reaches point B in 6 ...,2


In [4]:
rows_count = aptitude_questions_bank_df.shape[0]

In [5]:
# Extracting required sub categories 
required_columns = ['Finding Errors', 'Ratio and Proportion', 'Logarithms', 'Time and Distance', 'Simple and Compound Interest']
classified_questions_df = aptitude_questions_bank_df[aptitude_questions_bank_df['Sub-Category'].isin(required_columns)]
classified_questions_df['Sub-Category'].unique()

array(['Time and Distance', 'Finding Errors', 'Ratio and Proportion',
       'Simple and Compound Interest', 'Logarithms'], dtype=object)

## **Stage 2:** Data Pre-Processing

### 3 Marks -> Clean and Transform the data into a specified format

*   Remove the rows of the Questions column which contains blank / NaN.


*   Few set of questions have HTML tags within the question.
  - You can use Beautiful Soup library to convert HTML into text (Refer **"Dealing with HTML"** section from this [link](https://www.nltk.org/book/ch03.html).)


*  Consider Question column as feature and Sub-category as target variable. Convert Sub-category into numerical.

*  Drop the unwanted columns


  **Hint:** Use Label Encoder for obtaining a numeric representation, refer to the [link](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). 

In [6]:
# Get the column names
classified_questions_df.columns

Index(['Category', 'Sub-Category', 'Article', 'Questions', 'Answers'], dtype='object')

In [7]:
# Remove the rows of the Questions column which contains blank / NaN
nan_rows_count = classified_questions_df['Questions'].isna().sum()
print(nan_rows_count)
if nan_rows_count > 0:
  classified_questions_df = classified_questions_df[classified_questions_df['Questions'].notna()]

0


In [8]:
# YOUR CODE HERE for BeatifulSoup
classified_questions_df["Simplified_Question"] = [BeautifulSoup(question,'html.parser').get_text() for question in classified_questions_df['Questions']]

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() # DO NOT CHANGE THIS LINE as we will be using for the Test evaluation.

# YOUR CODE HERE for Fit label encoder and return encoded labels
X = classified_questions_df["Simplified_Question"]
classified_questions_df["Sub-Category"] = le.fit_transform(classified_questions_df["Sub-Category"])
y = classified_questions_df["Sub-Category"]

# Drop unwanted columns
classified_questions_df.drop(labels=['Category','Article', 'Questions', 'Answers'],axis=1,inplace=True)

In [10]:
# with pd.option_context("display.max_rows",None):
#   display(classified_questions_df)

In [11]:
# classified_questions_df = classified_questions_df.iloc[0:rows_count,:]

## **Stage 3:** Text representation using Bag of Words (BOW)

### 3 Marks -> a) Get valid words from all questions & add them to a list.


Treat each question as a separate document and get the list of words using the following:
1.   Split the sentence into words

2.   Remove Stop words. Use NLTK packages for getting the Stop words.

3.   Replace proper names with "name" 
  - Example: "Rahul" -> "name"
       
4.   Remove the single white space character (\n, \t, \f, \r), refer [link](https://developers.google.com/edu/python/regular-expressions)

5.   Ignore words whose length is less than 3 (Eg: 'is', 'a').

6.   Remove punctuation and non-alphabetic words

7.   Convert the text to lowercase

8.   Use the Porter Stemmer to normalize the words


Refer [link](https://www.nltk.org/book/ch03.html) for extracting the words.

Refer to the below links for more information.

1. [link1](https://medium.com/free-code-camp/an-introduction-to-bag-of-words-and-how-to-code-it-in-python-for-nlp-282e87a9da04)

2. [link2](https://dataaspirant.com/bag-of-words-bow/)

In [12]:
from nltk.tag import pos_tag

In [ ]:
porter = nltk.PorterStemmer() # Perform stemming of word

In [13]:
def perform_multiple_string_replacements(text):
  replacements = { 
        r'\s\t\f\r+' : r"",   # Single space white character
        r'\b\w{0,3}\b' : r"", # Words with length 0 to 3
        r'[^\w\s]' : r"", # Punctuation marks
        r'[^a-zA-Z]' : r"", # Not aplphabetic
        r"^\d+\s|\s\d+\s|\s\d+$" : r"" # Numeric values within strings
  }

  for key, value in replacements.items():
      text = re.sub(key, value, text)

  text.lower() # Lowercase the word

  text = porter.stem(text)

  return text

In [14]:
def extract_words_enhanced(question):
  # Split the sentence into words
  question_tokens = word_tokenize(str(question))

  # Remove punctuation from each question token
  table = str.maketrans('', '', string.punctuation)
  question_tokens = [w.translate(table) for w in question_tokens]
  question_tokens = [w for w in question_tokens if len(w) >= 1]
  question_tokens

  # Check non english words 
  
  # Remove Stop words. Use NLTK packages for getting the Stop words.
  stop = set(stopwords.words('english'))
  stop_words_removed = []
  question_tokens_without_sw = [word for word in question_tokens if not word in stopwords.words()]
  # print(question_tokens_without_sw)
  
  # Replace proper names with "name"
  tagged_sent = pos_tag(question_tokens_without_sw)
  question_tokens_after_nnp_cleanup = ["name" for word,pos in tagged_sent if pos == 'NNP'] + [word for word,pos in tagged_sent if pos != 'NNP'] 
  # print(question_tokens_after_nnp_cleanup)
  
  # Perform string cleanup operations
  tokens_after_string_cleanup = map(perform_multiple_string_replacements,question_tokens_after_nnp_cleanup)
  tokens_after_string_cleanup = list(tokens_after_string_cleanup)

  # Remove if empty string exists
  if "" in tokens_after_string_cleanup:
    tokens_after_string_cleanup.remove("")
  return tokens_after_string_cleanup

In [15]:
def tokenize(allquestions):
  valid_words = []
  for question in allquestions:
    # words = extract_words(question)
    words = extract_words_enhanced(question)
    valid_words.extend(words)
  return set(valid_words)

In [16]:
# Use the function to extract words for all questions
all_valid_words = list(tokenize(list(classified_questions_df["Simplified_Question"])))
if "" in all_valid_words:
    all_valid_words.remove("")
# print(all_valid_words)

### 4 Marks -> b) Generate vectors that can be used by the machine learning algorithm


1.   The length of the vector for each question will be the length of the valid words. Initialize each vector with all Zeros

2.   Compare each valid word with the words in question and generate the vectors based on the counter frequency of the word in that question.



In [17]:
def create_unique_word_dictionary(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dictionary = {}
    for i, word in enumerate(words):
        unique_word_dictionary.update({
            word: i
        })

    return unique_word_dictionary 

In [18]:
# Prepare dictionary of all unique words and create indexes for each word for one hot encoding
words_from_questions_corpus_dict = create_unique_word_dictionary(all_valid_words)

In [19]:
def generate_vectors(question):
    # Initialize each vector with all zeros. 
    bag_vector = np.zeros(len(all_valid_words))

    # Extracting words for each question and count the words
    words = extract_words_enhanced(question)
    word_dict = Counter(words)
    # print("Word dictionary:",word_dict)
    
    # YOUR CODE HERE 
    # Hint: If the word is in valid words then generate the vectors based on the counter frequency of the word in that question.
    for unique_word_in_current_question,freq_counter in word_dict.items():
      # print(unique_word_in_current_question,",",freq_counter)
      if len(unique_word_in_current_question)>0:
        for i,word in enumerate(words_from_questions_corpus_dict):                
          if word == unique_word_in_current_question:                     
            bag_vector[i] += freq_counter                    
    
    return np.array(bag_vector)

# first_que = str(classified_questions_df["Simplified_Question"].iloc[:1])
# print("{0}\n{1}\n".format(first_que,generate_vectors(first_que))) 

In [20]:
# Use the above function for collecting the vectors of all questions into a list.
# YOUR CODE HERE
def collect_all_questions_vectors(allquestions):
  questions_vectors = []
  for question in allquestions:
    current_question_vector = generate_vectors(question)
    questions_vectors.append(current_question_vector)
  return questions_vectors

In [21]:
# Use the function to extract words for all questions
all_questions_vectors_arrays_list = collect_all_questions_vectors(classified_questions_df["Simplified_Question"])
# print(all_questions_vectors_arrays_list)

In [22]:
# Combine all the list of vectors and create a dataframe with keys as columns
questions_unique_words_df_columns = list(words_from_questions_corpus_dict.keys())

In [23]:
# Prepare a dataframe with all bag of words count frequency as rows , words as column names
bag_of_words_keywords_df = pd.DataFrame()
for current_vector in all_questions_vectors_arrays_list:
  SR_vector = pd.Series(current_vector)
  bag_of_words_keywords_df = bag_of_words_keywords_df.append(SR_vector,ignore_index=True)

bag_of_words_keywords_df.columns = questions_unique_words_df_columns
bag_of_words_keywords_df.reset_index(inplace=True)

In [24]:
with pd.option_context("display.max_columns",None):
  display(bag_of_words_keywords_df)

index   ab  abab  abc  abil  abl  abnorm  absenc  absent  abund  accept  \
0         0  0.0   0.0  0.0   0.0  0.0     0.0     0.0     0.0    0.0     0.0   
1         1  0.0   0.0  0.0   0.0  0.0     0.0     0.0     0.0    0.0     0.0   
2         2  0.0   0.0  0.0   0.0  0.0     0.0     0.0     0.0    0.0     0.0   
3         3  0.0   0.0  0.0   0.0  0.0     0.0     0.0     0.0    0.0     0.0   
4         4  0.0   0.0  0.0   0.0  0.0     0.0     0.0     0.0    0.0     0.0   
...     ...  ...   ...  ...   ...  ...     ...     ...     ...    ...     ...   
1602   1602  0.0   0.0  0.0   0.0  0.0     0.0     0.0     0.0    0.0     0.0   
1603   1603  0.0   0.0  0.0   0.0  1.0     0.0     0.0     0.0    0.0     0.0   
1604   1604  0.0   0.0  0.0   0.0  0.0     0.0     0.0     0.0    0.0     0.0   
1605   1605  0.0   0.0  0.0   0.0  0.0     0.0     0.0     0.0    0.0     0.0   
1606   1606  0.0   0.0  0.0   0.0  0.0     0.0     0.0     0.0    0.0     0.0   

      accid  accompani  accord  accordingli  account  accru  accumul  accur  \
0       0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   
1       0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   
2       0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   
3       0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   
4       0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   
...     ...        ...     ...          ...      ...    ...      ...    ...   
1602    0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   
1603    0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   
1604    0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   
1605    0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   
1606    0.0        0.0     0.0          0.0      0.0    0.0      0.0    0.0   

      accus  achiev  acid  acquir  acquit  across  act  activ  actual   ad  \
0       0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   
1       0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   
2       0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   
3       0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   
4       0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   
...     ...     ...   ...     ...     ...     ...  ...    ...     ...  ...   
1602    0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   
1603    0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   
1604    0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   
1605    0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   
1606    0.0     0.0   0.0     0.0     0.0     0.0  0.0    0.0     0.0  0.0   

      addict  addit  address  adequ  administr  admir  admiss  admit  adopt  \
0        0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   
1        0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   
2        0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   
3        0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   
4        0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   
...      ...    ...      ...    ...        ...    ...     ...    ...    ...   
1602     0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   
1603     0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   
1604     0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   
1605     0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   
1606     0.0    0.0      0.0    0.0        0.0    0.0     0.0    0.0    0.0   

      adulter  advanc  advantag  advertis  advic  advis  advisor  aero  \
0         0.0     0.0       0.0       0.0    0.0    0.0      0.0   0.0   
1         0.0     0.0       0.0       0.0    0.0  

In [25]:
bag_of_words_keywords_df.drop(columns=["index"],inplace=True)

## **Stage 4:** Classification

### 3 Marks -> Perform a Classification 

1.   Identify the features and labels

2.   Use train_test_split for splitting the train and test data

3.   Fit your model on the train set using fit() and perform prediction on the test set using predict()

4. Get the accuracy of the model

## Expected Accuracy above 90%


In [26]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X = bag_of_words_keywords_df
y = classified_questions_df['Sub-Category']

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=20)
print("shape of X_train:",X_train.shape)
print("shape of y_train:",y_train.shape)
print("shape of X_test:",X_test.shape)
print("shape of y_test:",y_test.shape)

# Validation should be done

shape of X_train: (1285, 2228)
shape of y_train: (1285,)
shape of X_test: (322, 2228)
shape of y_test: (322,)


In [28]:
for d in np.arange(3,15):
  dr = DecisionTreeClassifier(criterion="gini",max_depth=d)
  dr.fit(X_train,y_train)
  y_pred = dr.predict(X_test)
  print("Depth = ",d,"Accuracy score = ",accuracy_score(y_pred,y_test))

Depth =  3 Accuracy score =  0.7639751552795031
Depth =  4 Accuracy score =  0.7857142857142857
Depth =  5 Accuracy score =  0.8074534161490683
Depth =  6 Accuracy score =  0.8291925465838509
Depth =  7 Accuracy score =  0.8229813664596274
Depth =  8 Accuracy score =  0.8385093167701864
Depth =  9 Accuracy score =  0.8571428571428571
Depth =  10 Accuracy score =  0.8819875776397516
Depth =  11 Accuracy score =  0.8819875776397516
Depth =  12 Accuracy score =  0.8788819875776398
Depth =  13 Accuracy score =  0.8819875776397516
Depth =  14 Accuracy score =  0.8788819875776398


In [29]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=20)
print("shape of X_train:",X_train.shape)
print("shape of y_train:",y_train.shape)
print("shape of X_test:",X_test.shape)
print("shape of y_test:",y_test.shape)

shape of X_train: (1285, 2228)
shape of y_train: (1285,)
shape of X_test: (322, 2228)
shape of y_test: (322,)


In [30]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [31]:
# Model Class to be used for different ML algorithms
class ClassifierModel(object):
    def __init__(self, clf, params=None):
        self.clf = clf(**params)
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    def fit(self,x,y):
        return self.clf.fit(x,y)
    def feature_importances(self,x,y):
        return self.clf.fit(x,y).feature_importances_
    def predict(self, x):
        return self.clf.predict(x)

In [32]:
def trainModel(model, x_train, y_train, x_test, n_folds, seed, shuffle_flag=False):
        cv = KFold(n_splits= n_folds, random_state=seed,shuffle = shuffle_flag)
        scores = cross_val_score(model.clf, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
        return scores

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
# Random Forest parameters
rf_params = {
    'n_estimators': 400,
    'max_depth': 5,
    'min_samples_leaf': 3,
    'max_features' : 'sqrt',
}
for d in np.arange(5,13):
  rf_params["max_depth"] = d
  rf_params["min_samples_leaf"] = d - 2
  rfc_model = ClassifierModel(clf=RandomForestClassifier, params=rf_params)
  rfc_scores = trainModel(rfc_model,X_train, y_train, X_test, 5, None,True)
  print("Max depth = ",d,",Samples lead",d-2,"scores : ", rfc_scores)

Max depth =  5 ,Samples lead 3 scores :  [0.83657588 0.84824903 0.89105058 0.80544747 0.85992218]
Max depth =  6 ,Samples lead 4 scores :  [0.84046693 0.84435798 0.87159533 0.89105058 0.87548638]
Max depth =  7 ,Samples lead 5 scores :  [0.85214008 0.87937743 0.84046693 0.88715953 0.88326848]
Max depth =  8 ,Samples lead 6 scores :  [0.86381323 0.86770428 0.88715953 0.89494163 0.85214008]
Max depth =  9 ,Samples lead 7 scores :  [0.84435798 0.87937743 0.87159533 0.87937743 0.87159533]
Max depth =  10 ,Samples lead 8 scores :  [0.85992218 0.90272374 0.85603113 0.87159533 0.85603113]
Max depth =  11 ,Samples lead 9 scores :  [0.87937743 0.89494163 0.81712062 0.85992218 0.85992218]
Max depth =  12 ,Samples lead 10 scores :  [0.89883268 0.84046693 0.82490272 0.86381323 0.82490272]


In [35]:
from sklearn.ensemble import ExtraTreesClassifier

In [36]:
# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':400,
    'max_depth': 11,
    'min_samples_leaf': 2,
}
etc_model = ClassifierModel(clf=ExtraTreesClassifier, params=et_params)
etc_model.fit(X_train,y_train)

etc_scores = trainModel(etc_model,X_train, y_train, X_test, 5, None , True) # Random Forest
etc_scores

array([0.88326848, 0.85992218, 0.88326848, 0.84046693, 0.84824903])

In [37]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [38]:
# Voting Classifier
algorithms_to_use = []
algorithms_to_use.append(('lr',LogisticRegression(max_iter=200)))
algorithms_to_use.append(('dt',DecisionTreeClassifier()))
algorithms_to_use.append(('sv',SVC(probability=True)))

voting_cf_hard = VotingClassifier(estimators=algorithms_to_use,voting="hard")
voting_cf_hard.fit(X_train,y_train)
y_pred = voting_cf_hard.predict(X_test)

# Print the accuracy score
print(accuracy_score(y_test,y_pred))

0.9440993788819876


In [39]:
from sklearn.ensemble import BaggingClassifier

In [40]:
# Bagging classifier parameters
knn = KNeighborsClassifier(3)
dt = DecisionTreeClassifier(max_depth=2)
svm = SVC()

classifiers = [("KNN", knn), ("Decision_Tree", dt), ("SVM", svm)]

# Using different classifiers as base_estimator
for clf_name, clf in classifiers:
    # Instantiate bagging classifier
    model = BaggingClassifier(base_estimator = clf, bootstrap=True)
    
    # YOUR CODE HERE: To fit model on training dataset
    model.fit(X_train,y_train)

    # YOUR CODE HERE: To get the predictions on test dataset
    y_pred = model.predict(X_test)

    # Evaluate the accuracy of clf on the test set
    print("{:s} : {:.3f}".format(clf_name, accuracy_score(y_test, y_pred)))

KNN : 0.792
Decision_Tree : 0.693
SVM : 0.925


In [41]:
pd.DataFrame(X).to_csv("final_related_data.csv")

## **Stage 5:** Evaluation (This is for Mentors)

### 6 Marks -> Evaluate with the given test data 

1.  Loading the Test data

2.  Converting the Test data into vectors

3.  Pass through the model and verify the accuracy

## Expected Accuracy above 90%


In [42]:
# YOUR CODE HERE for selecting the trained classifier model, eg: MODEL = decision_tree
MODEL = voting_cf_hard

Test_data = pd.read_csv("Mentors_Test_Data.csv")
Test_data = Test_data[Test_data['Sub-Category'].isin(le.classes_)]
labels = le.transform(Test_data['Sub-Category'])
Test_questions= Test_data['Questions']

Test_BOW=[]
for TQ in Test_questions: 
  Test_vectors = generate_vectors(TQ) 
  Test_BOW.append(Test_vectors)

predict = MODEL.predict(Test_BOW) 
accuracy_score(labels, predict)

0.9325842696629213